# Find Raw Data from Folders of NWB files

### from a folder of NWB files like this:

```
nwb-files-folder/
    2023_04_20-13-06-20.nwb
    2023_04_20-15-26-40.nwb
    2023_04_20-17-06-20.nwb
```

### It will generate a script, that you can execute in your raw data folder:

```
raw/
    TSeries-04202023-001/
    TSeries-04202023-002/
    TSeries-04202023-003/
    TSeries-04202023-004/
    TSeries-04202023-005/
    2023_04_20/13-06-20/
    2023_04_20/14-28-40/
    2023_04_20/15-26-40/
    2023_04_20/17-06-20/
```

### to put all the raw data related to the NWB files in a folder `to-keep/`

```
raw/
    TSeries-04202023-002/
    TSeries-04202023-004/
    2023_04_20/14-28-40/
    to-keep/
        2023_04_20/
            TSeries-04202023-001/
            TSeries-04202023-003/
            TSeries-04202023-005/
            13-06-20/
            15-26-40/
            17-06-20/
```

In [2]:
import sys
sys.path.append('../../..')
import physion

import warnings
warnings.filterwarnings("ignore") # disable the UserWarning from pynwb (arrays are not well oriented)

In [3]:

def find_folder_infos(datafolder, 
                      Nmax=1000,
                      verbose=True):

    DATASET = physion.analysis.read_NWB.scan_folder_for_NWBfiles(datafolder)

    TSeries, DayFolders, TimeFolders = [], [], []
    for i, filename in enumerate(DATASET['files'][:Nmax]):

        if verbose:
            print('- %s' % filename)
        data = physion.analysis.read_NWB.Data(filename, verbose=verbose)

        Description = str(data.nwbfile.processing['ophys'].description)
        # print('     * %s' % Description)
        TSeries.append('TSeries-'+Description.split('TSeries-')[2].split('/')[0])
        if verbose:
            print('     * TSeries-folder: %s' % TSeries[-1])
        DayFolders.append(str(data.nwbfile.identifier)[:10])
        TimeFolders.append(str(data.nwbfile.identifier)[11:])
        if verbose:
            print('     * Day-folder: %s' % DayFolders[-1])
            print('     * Time-folder: %s' % TimeFolders[-1])

    return TSeries, DayFolders, TimeFolders



def build_bash_script(datafolder, new_folder, 
                      Nmax=1000,
                      verbose=True):

    script = """
if ! [ -d ./{folder} ]; then mkdir ./{folder} ; fi

move_to_keep() {{

    if ! [ -d ./{folder}/$2 ]; then mkdir ./{folder}/$2; fi

    # TSeries folder first
    if test -d $1; then
        {{  
        mv $1 ./{folder}/$2/
        echo $1 \" moved to ./{folder}/\"$2
        }}
    else
        {{
        echo \"\"
        echo \" /!\\" $1 \" folder not found !!\"
        echo \"\"
        }}
    fi

    # Then VisualStim+Behavior
    if test -d $2/$3; then
        {{  
        mv $2/$3 ./{folder}/$2/
        echo $2/$3 \" moved to ./{folder}/\"$2
        }}
    else
        {{
        echo \"\"
        echo \" /!\\" $2/$3 \" folder not found !!\"
        echo \"\"
        }}
    fi
}}
""".format(**{'folder':new_folder})

    TSeries, DayFolders, TimeFolders = find_folder_infos(datafolder, Nmax=Nmax, verbose=verbose)

    for TSerie, DayFolder, TimeFolder in zip(TSeries, DayFolders, TimeFolders):
        
        script += 'move_to_keep %s %s %s \n' % (TSerie, DayFolder, TimeFolder)

    return script

In [4]:
datafolder = os.path.join(os.path.expanduser('~') ,'DATA', 'SST-FF-Gratings-Stim')

new_folder = 'to-keep'
# TSeries, DayFolders, TimeFolders = find_folder_infos(datafolder)
script = build_bash_script(datafolder, new_folder, Nmax=1000, verbose=False)

inspecting the folder "/Users/yann/DATA/SST-FF-Gratings-Stim/" [...]
 -> found n=32 datafiles (in 2.0s) 


In [5]:
# write as a bash script
script_name = 'archive.sh'
with open(script_name, 'w') as f:
    f.write(script)

In [6]:
cat archive.sh


if ! [ -d ./to-keep ]; then mkdir ./to-keep ; fi

move_to_keep() {

    if ! [ -d ./to-keep/$2 ]; then mkdir ./to-keep/$2; fi

    # TSeries folder first
    if test -d $1; then
        {  
        mv $1 ./to-keep/$2/
        echo $1 " moved to ./to-keep/"$2
        }
    else
        {
        echo ""
        echo " /!\" $1 " folder not found !!"
        echo ""
        }
    fi

    # Then VisualStim+Behavior
    if test -d $2/$3; then
        {  
        mv $2/$3 ./to-keep/$2/
        echo $2/$3 " moved to ./to-keep/"$2
        }
    else
        {
        echo ""
        echo " /!\" $2/$3 " folder not found !!"
        echo ""
        }
    fi
}
move_to_keep TSeries-02152023-1058-001 2023_02_15 11-53-39 
move_to_keep TSeries-02152023-1058-002 2023_02_15 12-41-21 
move_to_keep TSeries-02152023-1058-007 2023_02_15 16-40-50 
move_to_keep TSeries-02152023-1058-008 2023_02_15 17-14-56 
move_to_keep TSeries-02152023-1058-009 2023_02_15 18-05-25 
move_to_keep TSeries-02152023-1058-010 20